In [83]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

In [103]:
columns = ['sample_code', 'clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity', 'marginal_adhesion', 'single_epithelial_cell_size',
             'bare_nuclei', 'bland_chromatin', 'normal_nucleoli', 'mitoses', 'class']
data = pd.read_csv("../data/logistic_regression/breast-cancer-wisconsin.data", header=None, names=columns, na_values=[np.nan, '?'])
data = data.fillna(data.median())

In [104]:
np.random.seed(1)
train = data.sample(frac=0.8).copy()
y_train = (train['class'] == 4).astype(int)
train.drop(['sample_code', 'class'], axis=1, inplace=True)

test = data.loc[~data.index.isin(train.index)].copy()
y_test = (test['class'] == 4).astype(int)
test.drop(['sample_code', 'class'], axis=1, inplace=True)

In [105]:
def create_logreg(feature_columns, feature_layer_inputs, optimizer, loss='binary_crossentropy', metrics=['accuracy'], l2=0.01):
    regularizer = keras.regularizers.l2(l2)
    feature_layer = keras.layers.DenseFeatures(feature_columns)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    norm = keras.layers.BatchNormalization()(feature_layer_outputs)
    outputs = keras.layers.Dense(1, kernel_initializer='normal', kernel_regularizer=regularizer, activation='sigmoid')(norm)
    
    model = keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=outputs)
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    return model
    

In [106]:
def define_feature_column_layers(data, categorical_cols, numeric_cols):
    feature_columns = list()
    feature_layer_inputs = dict()
    
    for feature_name in numeric_cols:
        feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name)
        
    for feature_name in categorical_cols:
        vocabulary = data[feature_name].unique()
        cat = tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary)
        cat_one_hot = tf.feature_column.indicator_column(cat)
        feature_columns.append(cat_one_hot)
        feature_layer_inputs[feature_name] = tf.keras.Input(shape=(1,), name=feature_name, dtype=tf.int32)
        
    return feature_columns, feature_layer_inputs

In [107]:
def make_input_fn(data, label, num_epochs=10, shuffle=True, batch_size=256):
    def input_function():
        ds = tf.data.Dataset.from_tensor_slices((dict(data), label))
        if shuffle:
            ds = ds.shuffle(1000)
        
        ds = ds.batch(batch_size).repeat(num_epochs)
        return ds
    return input_function

In [108]:
import tempfile

def canned_keras(model):
    model_dir = tempfile.mkdtemp()
    keras_estimator = keras.estimator.model_to_estimator(keras_model=model, model_dir=model_dir)
    return keras_estimator

In [109]:
categorical_cols = []
numeric_cols = ['clump_thickness', 'cell_size_uniformity', 'cell_shape_uniformity','marginal_adhesion', 'single_epithelial_cell_size', 
                 'bare_nuclei', 'bland_chromatin','normal_nucleoli', 'mitoses']

feature_columns, feature_layer_inputs = define_feature_column_layers(data, categorical_cols, numeric_cols)
optimizer = keras.optimizers.Ftrl(learning_rate=0.007)
model = create_logreg(feature_columns, feature_layer_inputs, optimizer)
estimator = canned_keras(model)
train_input_fn = make_input_fn(train, y_train, num_epochs=300, batch_size=32)
test_input_fn = make_input_fn(test, y_test, num_epochs=1, shuffle=False)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.


C:\Users\Siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Siddh\\AppData\\Local\\Temp\\tmpb527bf71', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [110]:
estimator.train(train_input_fn)

INFO:tensorflow:Calling model_fn.


C:\Users\Siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='C:\\Users\\Siddh\\AppData\\Local\\Temp\\tmpb527bf71\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: C:\Users\Siddh\AppData\Local\Temp\tmpb527bf71\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 4 variables.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Siddh\AppData\Local\Temp\tmpb527bf71\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.674492, step = 0
INFO:tensorflow:global_step/

In [113]:
estimator.evaluate(test_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-10-31T15:28:58
INFO:tensorflow:Graph was finalized.


C:\Users\Siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
C:\Users\Siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


INFO:tensorflow:Restoring parameters from C:\Users\Siddh\AppData\Local\Temp\tmpb527bf71\model.ckpt-5400
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.17621s
INFO:tensorflow:Finished evaluation at 2022-10-31-15:28:59
INFO:tensorflow:Saving dict for global step 5400: accuracy = 0.95, global_step = 5400, loss = 0.16435947
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5400: C:\Users\Siddh\AppData\Local\Temp\tmpb527bf71\model.ckpt-5400


{'accuracy': 0.95, 'loss': 0.16435947, 'global_step': 5400}